# Rapport DM TAL

## Analyse des prétraitements

### Présidents

L'analyse des prétraitements du text c'est fait en faisant varier les paramètres de CountVectorizer et TfidfVectorizer. Les fonctions de prétraitements de text on aussi été recodé mais par volonté de rapidité on a fait varier ces paramètres (excepté la lemmatisation) en faisant appel drectement à gridsearchVC. Voici la liste des paramètres testés (une cross validatio a été fixé à 5):

```
tf_params = {
    'tvec__lowercase': ['True', 'False'],
    'tvec__analyzer' : ['word', 'char', 'char_wb'],
    'tvec__strip_accents': [None, 'ascii'],
    'tvec__stop_words': [None, stop1, stop2, stop3],
    'tvec__min_df': [0.1,0.2],
    'tvec__max_df': [0.2,0.5,0.9],
    'tvec__max_features': [10, 100 ,1000],
    'tvec__ngram_range': [(1, 1), (1,3), (1,4)],
}
```

Les différentes listes de stop word (stop1,2,3) correspond à la liste des stop words optenus à partir de la fréquence documentaire (1), la fréquence tf-idf (2) et les stop word avec un ngram de 1,4 (3). \

#### Modèle Bayesien

* Prétraitement sélectionnés :
```
{'analyzer': 'word', 'lowercase': 'True',  'max_df': 0.9, 'max_features': 100, 
'min_df': 0.1,  'ngram_range': (1, 1), 'stop_words': None, 'strip_accents': 'ascii'}
 ```

#### Modèle Regression Logisitique

* Prétraitement sélectionnés :
```
{'analyzer': 'word', 'lowercase': 'True', 'max_df': 0.5, 'max_features': 100, 'min_df': 0.1,
 'ngram_range': (1, 1), 'stop_words': None, 'strip_accents': 'ascii'}
```

#### Modèle SVM

* Prétraitement sélectionnés :
```
{'analyzer': 'word', 'lowercase': 'True', 'max_df': 0.5, 'max_features': 1000,
 'min_df': 0.1, 'ngram_range': (1, 1), 'stop_words': None, 'strip_accents': 'ascii'}
```

* Scores :

| Données | Bayesien | Regression | SVM |
| -------- | -------- | -------- | -------- |
| Train | 0.8698 | 0.8709 | 0.8703 |
| Test | 0.8884 | 0.8877 | 0.8903 | 


* Détail des tests :

<img src="./output/president_pretait_1.png" alt="pretraitement_prez" />


A partir de ces premiers résultats que d'une part les modèles sont relativement proches en terme de performance et que la régression logistique, même si elle semble donner de meilleur résultat sur les données d'entrainement, donne des performances moins bonnes sur les données test.

On renouvelle l'experience mais cette fois-ci avec une lemmatisation du text en amont.

| Données | Bayesien | Regression | SVM |
| -------- | -------- | -------- | -------- |
| Train | 0.8694 | 0.8706 | 0.8706 |
| Test | 0.8883 | 0.8876 | 0.8908 |

La lemmatisation ne semble pas impacter grandement les performances. Maintenant renouvellons les tests mais avec tous les prétraitements (sauf la lemmatisation).

| Données | Bayesien | Regression | SVM |
| -------- | -------- | -------- | -------- |
| Train | 0.8698 | 0.8709 | 0.8703 |
| Test | 0.8889 | 0.8877 | 0.8903 |

### Movies

On a fait varier chaque paramètres en prenant les valeurs par défault pour les aurtes paramètres pour chacun des trois modèles (Naives bayes, Regression et SVM). \

<img src="./output/movies_pretrait.png" alt="pretraitement_movies" />

Cette approche est plus économe en terme de temps et permet aussi de se rendre compte plus facilement de l'impact de certains choix sur les performances. On peut notamment voir que gloablement le modèle bayesien est moins efficace pour notre exercice. De meme, le choix du bon grammage aura un impacte significatif sur la performance des modèles.

## Fine tunning des paramètres des modèles


### Président

| Données | Regression | SVM | Random-Forest |
| -------- | -------- | -------- | -------- |
| Train | 0.87222 | 0.8715 | 0.8689 |
| Test | 0.8848 | 0.8882 | 0.8925 |

* Regression linéaires :

```
LG_params = {
            'C': [1e-5, 0.001, 0.01, 0.1, 0.5, 0.9],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg' ],
            'class_weight':[None, 'balanced'], 
            'max_iter': [100,1000] 
    }
```

Le score en train augmente mais celui en test diminu. On commence à faire de l'overfitting du le modèle.


* SVM :

```
LSVC_params = {
            'C':[0.00001, 0.0001, 0.001],
            'class_weight':[None, 'balanced'], 
            'penalty': ['l1', 'l2'],
            'loss' : ['hinge', 'squared_hinge'],
    }
```

Même constat que pour la régression linéaire, une modification des hyperparamètres du modèle enrtaîne un sur-apprentissage.

* Random-Forest : 

```
RF_params = {
            'n_estimators': [10, 50, 100], 
            'criterion': ['entropy'],
            'max_features': ['auto', 'sqrt', 'log2'], 
            'max_depth': [3, 5, 7, 10],
            'bootstrap': [True],
            'class_weight': [None, 'balanced', 'balanced_subsample'],
            'ccp_alpha': [0.001, 0.01, 0.1, 1, 10]
    }
```
Le random forest semble être une technique mieux adaptée. On obtient un résultat plus performant en test que les précédents modèles

## Movies 

A partir des résultats précédents on a essayé de combiner les différents prétraitements pour améliorer nos performances. \

<img src="./output/movies_pretrait2.png" alt="mvs2" />

## Equilibrage des données

### Président

Le jeu de données initial est désequilibré comme le montre ce graphique : 

<img src="./output/president_classes.png" alt="classes_prez" />

On a environ 85 % des données qui sont de la classe 1. Donc nécessairement, notre modèle sera beaucoup plus à même de trouver d'identifier les élèments de la classe 1 par rapport à la classe -1.
On a effectué différents types d'approches pour essayer de combler ce problème :
* Ré-équilibrer le jeu de données en supprimant des données dans la classe majoritaire
* Attribuer différents poids aux classes.
* Utiliser des classifieurs plus robustes face au problème d'équilibrage comme le gradient boosting.


Cependant, aucune de ces approches n'a été très convaincantes. Le gradient de boosting obtient effectivement un score plus élévé que les autres classifieurs (0.905944646682807) pour les données d'entrainement mais il est moins performant sur les données de test (0.8654370075841249).


## Conclusion

### Président

Après ce travail d'analyse, nous décidons de partir sur un classifieur de type Random Forest avec les prétraitement suivant :
* Pas de majuscules + Pas de ponctuations + Pas de chiffres
* Convertorizer : Ngram (1,1) + Pas de stop word
* Hyper-paramètre : bootstrap =  True, ccp_alpha = 0.001, class_weight = None, criterion = entropy,max_depth = 10, max_features = sqrt, n_estimators = 10

Score :
* Train : 0.8689669587027329
* Test : 0.8925705029084751

L'exercice n'est pas vraiment bien réussi puisque lorsqu'on regarde dans le détail des prédictions en réalité random forest c'est juste contenté de tout classifer comme étant 1...

## Movies

D'après nos résultats les prétraitements les plus optimisés pour chacun de nos modèles testés sont les suivantes : \
* Max NB :  Stemming + Sans stop words + (1,3)gram
* Max LR :  Sans stop words + Sans ponctuation + Unigram
* Max SVM :  Sans chiffres + (1,3)gram

On sélection donne le classifieur SVM avec un prétraitement sans chiffre et un grammage de (1,3)